In [85]:
import os
import pickle
#import tomotopy as tp
import pandas as pd
from dateutil import parser
from langdetect import detect, DetectorFactory
import ast

In [139]:
import runtimex
import importlib
importlib.reload(runtimex)

<module 'runtimex' from '/home/zly/Luyue/Luyue/runtimex.py'>

In [4]:
article_list = []
with open("poynter_crawl_overall_list.pkl", "rb") as f:
    article_list = pickle.load(f)
with open("poynter_crawl_overall_list_2.pkl", "rb") as f:
    article_list_2 = pickle.load(f)
article_list.extend(article_list_2)
print("There are " + str(len(article_list)) + " articles in the database")


There are 19929 articles in the database


In [5]:
#We examine the publish date of the articles, removing those from after March. 25th, 2020
new_article_list = []
for each_article in article_list:
    publish_date_str = each_article[1]
    #print(publish_date)
    try:
        publish_date = parser.parse(publish_date_str)
        if publish_date.replace(tzinfo=None) < parser.parse("2020-03-26 00:00:00+00:00").replace(tzinfo=None):
            new_article_list.append(each_article)
    except Exception as e:
        #If there is no date we have to include it.
        #print(e)
        new_article_list.append(each_article)
print("There are now " + str(len(new_article_list)) + " articles published before 2020-03-26.")

There are now 9872 articles published before 2020-03-26.


In [10]:
DetectorFactory.seed = 0
english_list = []
for each_article in new_article_list:
    if each_article != '':
        try:
            if detect(each_article[4][0]) == 'en':
                english_list.append(each_article)
        except Exception as e:
            pass

print("There are " + str(len(english_list)) + " english articles in database.")

There are 2978 english articles in database.


In [34]:
os.environ["TOKENIZERS_PARALLELISM"] = "true"
import torch.multiprocessing as multiprocessing
if multiprocessing.get_start_method() == 'fork':
    multiprocessing.set_start_method('spawn', force=True)
process_list = []
device_num = 0
corpus_main = "/home/zly/Luyue/Luyue/Extractive_summary_no_retweet/"
for each_keyword in os.listdir(corpus_main):
    p = multiprocessing.Process(target=runtimex.Extraction_comparison_of_keyword, args=(each_keyword, device_num, english_list))
    device_num += 1
    if device_num >= 8:
        device_num = 0
    process_list.append(p)

for p in process_list:
    p.start()
for p in process_list:
    p.join()

In [52]:
os.environ["TOKENIZERS_PARALLELISM"] = "true"
import torch.multiprocessing as multiprocessing
if multiprocessing.get_start_method() == 'fork':
    multiprocessing.set_start_method('spawn', force=True)
process_list = []
device_num = 0
corpus_main = "/home/zly/Luyue/Luyue/Extractive_summary_no_retweet/"
for each_keyword in os.listdir(corpus_main):
    p = multiprocessing.Process(target=runtimex.Dup_removal, args=(each_keyword,))
    device_num += 1
    if device_num >= 8:
        device_num = 0
    process_list.append(p)

for p in process_list:
    p.start()
for p in process_list:
    p.join()

Finished government date 2020-01-23
Finished government date 2020-03-20
Finished government date 2019-10-22
Finished government date 2020-02-05
Finished government date 2019-10-02
Finished government date 2020-03-21
Finished government date 2019-10-01
Finished government date 2019-10-11
Finished government date 2019-11-25
Finished government date 2019-10-23
Finished government date 2020-01-09
Finished government date 2019-12-23
Finished government date 2019-12-31
Finished government date 2019-11-28
Finished government date 2019-12-16
Finished government date 2020-02-22
Finished government date 2019-10-24
Finished government date 2020-01-06
Finished government date 2020-02-21
Finished government date 2020-03-14
Finished government date 2020-01-21
Finished government date 2019-11-29
Finished government date 2020-03-13
Finished government date 2019-12-27
Finished government date 2020-02-09
Finished government date 2019-12-30
Finished government date 2019-12-14
Finished government date 201

In [147]:
os.environ["TOKENIZERS_PARALLELISM"] = "true"
import torch.multiprocessing as multiprocessing
if multiprocessing.get_start_method() == 'fork':
    multiprocessing.set_start_method('spawn', force=True)
process_list = []
device_num = 0
corpus_main = "/home/zly/Luyue/Luyue/Extractive_summary_no_retweet/"
for each_keyword in os.listdir(corpus_main):
    p = multiprocessing.Process(target=runtimex.Comparison_analysis, args=(each_keyword,))
    device_num += 1
    if device_num >= 8:
        device_num = 0
    process_list.append(p)

for p in process_list:
    p.start()
for p in process_list:
    p.join()


In [68]:
overall_similarity = list(df["6"])
thresholds = [0.65, 0.6, 0.55, 0.5, 0.45, 0.4, 0.35, 0.3]
for threshold in thresholds:
    count = sum(1 for number in overall_similarity if number > threshold)
    print(f'Numbers greater than {threshold}: {count}')

Numbers greater than 0.65: 14
Numbers greater than 0.6: 42
Numbers greater than 0.55: 102
Numbers greater than 0.5: 182
Numbers greater than 0.45: 316
Numbers greater than 0.4: 534
Numbers greater than 0.35: 809
Numbers greater than 0.3: 1047


In [148]:
stat_dir = "/home/zly/Luyue/Luyue/Final_label_statistics/"
combined_df = pd.DataFrame()
for keyword in os.listdir(stat_dir):
    keyword_dir = os.path.join(stat_dir, keyword)
    file_name = keyword_dir + "/output_allthreshold.csv"
    current_df = pd.read_csv(file_name)
    combined_df = pd.concat([combined_df, current_df], ignore_index=True)

In [149]:
combined_df

,Keyword,Date,Topic,Count_list
0,lab,2020-01-23,10,"[0, 0, 0, 0, 0, 0, 1, 18, 80, 226, 417, 640, 8..."
1,lab,2020-01-23,21,"[0, 0, 0, 0, 0, 0, 6, 41, 129, 261, 458, 698, ..."
2,lab,2020-01-23,8,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, ..."
3,lab,2020-01-23,24,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 4, 30, 77, 219, 49..."
4,lab,2020-01-23,16,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12, 44, 192, 46..."
...,...,...,...,...
247795,spread,2020-02-11,7,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
247796,spread,2020-02-11,6,"[0, 0, 0, 0, 0, 0, 14, 55, 115, 210, 343, 556,..."
247797,spread,2020-02-11,18,"[0, 0, 0, 0, 0, 0, 14, 56, 142, 270, 455, 683,..."
247798,spread,2020-02-11,15,"[0, 0, 0, 0, 0, 0, 1, 7, 47, 136, 337, 589, 82..."


In [150]:
combined_df.to_csv("Comparison_Combined_Dataframe_all_threshold.csv")